# Explore here

A este Dataframe ya se le realizo el EDA en proyectos anteriores de 4Geeks (Arbol de Desicion)

In [1]:
# Data

import pandas as pd

df = pd.read_csv('/workspaces/Boosting_Algorithms/data/processed/df_diabetes_eda.csv', sep=';')
df

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,33.6,0.627,50,1
1,1,85,66,26.6,0.351,31,0
2,8,183,64,23.3,0.672,32,1
3,1,89,66,28.1,0.167,21,0
4,5,116,74,25.6,0.201,30,0
...,...,...,...,...,...,...,...
614,9,89,62,22.5,0.142,33,0
615,2,122,70,36.8,0.340,27,0
616,5,121,72,26.2,0.245,30,0
617,1,126,60,30.1,0.349,47,1


In [2]:
# Division de conjuntos Train y Test

from sklearn.model_selection import train_test_split

X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# Modelo y Entrenamiento

from xgboost import XGBClassifier

model = XGBClassifier(random_state=42)

model.fit(X_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [ ]:
# Predicciones

y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1])

In [ ]:
# Evaluacion del modelo

from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.782258064516129

In [9]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [50, 100, 200, 300],                  
    'learning_rate': [0.01, 0.1, 0.2, 0.001, 0.05],           
    'colsample_bytree': [0.6, 0.8, 1.0],   
    'gamma': [0, 1, 5, 8],                                 
}


grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',        
    cv=5,                    
    n_jobs=-1,        
)

grid_search.fit(X_train, y_train)


print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor score: {grid_search.best_score_}")

best_model = grid_search.best_estimator_


Mejores parámetros: {'colsample_bytree': 0.6, 'gamma': 8, 'learning_rate': 0.1, 'n_estimators': 50}
Mejor score: 0.8


In [10]:
# Predicciones best_model

y_best_pred = best_model.predict(X_test)
y_best_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1])

In [11]:
# Evaluacion del best_model

accuracy_score(y_test,y_best_pred)

0.8145161290322581

In [12]:
best_model.save_model('../models/xgbclassifier_csbt_0.6_gm_8_lr_0.1_ne_50_42.json')

El XGBClassifier tuvo un muy buen desempeno en este problema, sin embargo es un 0.005 peor que el RandomForest del proyecto anterior (Random Forest) pero mejor que el Arbol de Desicion simple, de todas formas se podria seguir manejando los hiperparametro para mejorarlo un poco mas.

Accuracy del Decission Tree: 0.7741935483870968
Accuracy del Random Forest:  0.8225806451612904
Accuracy del XGBClassifier : 0.8145161290322581

En este Problema, decido quedarme con el RandomForest, debido a que obtuvo un mejor resultado y aun se podria mejorar.

    Conclusion: Los modelos basados en arboles, como el RandomForest y XGBoost son muy buenos para este tipo de problemas, pudiendo manejar grandes cantidades de datos en tiempos bajos y gran cantidad de hiperparametros para su optimizacion obteniendo un buen resultado, sin embargo son mas complejos de explicar que un DecissionTree. Por lo tanto, si el problema requiere ser mas explicativo en el como se llego al resultado, es mejor elegir un DecissionTree, pero si necesitamos mayor accuracy en la prediccion sin importar su explicabilidad se debe utilizar un RandomForest o XGBoost

